taken from https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/translation.ipynb#scrollTo=545PP3o8IrJV

In [1]:
!pip install datasets trasformers[sentencepiece] sacrebleu

ERROR: Could not find a version that satisfies the requirement trasformers[sentencepiece] (from versions: none)
ERROR: No matching distribution found for trasformers[sentencepiece]


In [2]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to C:\Users\aonno/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


git LFS is installed from gitbash.

In [3]:
import transformers

print(transformers.__version__)

4.15.0


There is a different script for training multiple versions at github: https://github.com/huggingface/transformers/tree/main/examples/seq2seq

In [4]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-ro"

In [5]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("wmt16", "ro-en")
metric = load_metric("sacrebleu")

Reusing dataset wmt16 (C:\Users\aonno\.cache\huggingface\datasets\wmt16\ro-en\1.0.0\9e0038fe4cc117bd474d2774032cc133e355146ed0a47021b2040ca9db4645c0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 610320
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
})

In [7]:
raw_datasets["train"][0]

{'translation': {'en': 'Membership of Parliament: see Minutes',
  'ro': 'Componenţa Parlamentului: a se vedea procesul-verbal'}}

In [11]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Cant pick up any more examples"
    picks = []
    
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick =  random.randint(0, len(dataset) - 1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
        
    

In [12]:
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': '(FR) Madam President, cohesion policy must ensure harmonious and above all fair development between all territories.', 'ro': '(FR) Dnă președintă, politica de coeziune trebuie să asigure o dezvoltare armonioasă și mai ales echitabilă a tuturor teritoriilor.'}"
1,"{'en': 'When it comes to the votes, each country has a vote that matches its size.', 'ro': 'Referitor la vot, fiecare țară are un număr de voturi corespunzător dimensiunilor sale.'}"
2,"{'en': 'The budget in question has been cut by 2.3% in real terms when compared to the 2011 budget, and has suffered a total cut of EUR 48.9 million when compared with the proposed estimates.', 'ro': 'Bugetul în cauză a fost redus cu 2,3 % în termeni reali comparativ cu bugetul anului 2011, și a suferit o reducere totală de 48,9 milioane de euro comparativ cu estimările propuse.'}"
3,"{'en': 'Member of the Commission. - Madam President, very briefly a couple of points.', 'ro': 'membru al Comisiei. - Doamnă preşedintă, foarte pe scurt câteva puncte.'}"
4,"{'en': 'This text does not yet resolve this fundamental issue, which is why I decided to abstain from voting on the text.', 'ro': 'Textul încă nu rezolvă această chestiune fundamentală și de aceea am decis să mă abțin de la votarea acestuia.'}"


In [13]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: The system stream (a sequence of segments).
    references: A list of one or more reference streams (each a sequence of segments).
    smooth_method: The smoothing method to use. (Default: 'exp').
    smooth_value: The smoothing value. Only valid for 'floor' and 'add-k'. (Defaults: floor: 0.1, add-k: 1).
    tokenize: Tokenization method to use for BLEU. If not provided, defaults to 'zh' for Chinese, 'ja-mecab' for
        Japanese and '13a' (mteval) otherwise.
    lowercase: Lowercase the data. If True, enables case-insensitivity. (Default: False).
    force: Insist that your tokenized input is actually detokenized.

Returns:
    'score': BLEU score,
    'counts'

We have now loaded the dataset and metric

In [14]:
fake_preds = ["Hello", "General Kuboai"]
fake_labels = [["hello"], ["general kubiu"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [0, 0, 0, 0],
 'totals': [3, 1, 0, 0],
 'precisions': [0.0, 0.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 3,
 'ref_len': 3}

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers Tokenizer which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the AutoTokenizer.from_pretrained method, which will ensure:

we get a tokenizer that corresponds to the model architecture we want to use,
we download the vocabulary used when pretraining this specific checkpoint.
That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/770k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

language codes = https://huggingface.co/facebook/mbart-large-cc25

For the mBART tokenizer (like we have here), we need to set the source and target languages (so the texts are preprocessed properly). You can check the language codes here if you are using this notebook on a different pairs of languages.

In [17]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "en-XX"
    tokenizer.dst_lang = "ro-RO"

In [18]:
tokenizer("Hello this is one sentence")

{'input_ids': [125, 778, 63, 32, 141, 9191, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [19]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello this is one sentence.", "This is another sentence."]))

{'input_ids': [[10334, 1204, 15, 8915, 32, 27, 452, 59, 29579, 581, 2, 0], [235, 1705, 11, 32, 8, 1205, 5305, 59, 29579, 581, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [20]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the tokenizer with the argument truncation=True. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [24]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ro"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    target = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_target_length, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [25]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[1095, 5519, 11, 12785, 14, 4009, 1778, 1194, 53, 59, 35, 254, 30729, 11, 0], [9748, 3445, 3857, 14, 254, 30729, 11, 14, 513, 1315, 1807, 137, 25, 1182, 53, 59, 35, 254, 30729, 11, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[42140, 494, 1750, 53, 8, 59, 903, 3543, 9, 15202, 0], [36199, 6612, 9, 15202, 122, 568, 35788, 21549, 53, 8, 59, 903, 3543, 9, 15202, 0]]}

In [26]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/611 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass load_from_cache_file=False in the call to map to not use the cached files and force the preprocessing to be applied again.

Note that we passed batched=True to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

### Fine tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the AutoModelForSeq2SeqLM class. Like with the tokenizer, the from_pretrained method will download and cache the model for us.

In [29]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/287M [00:00<?, ?B/s]

In [31]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=True,
)

In [33]:
import torch
device = torch.device('cuda')

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the batch_size defined at the top of the cell and customize the weight decay. Since the Seq2SeqTrainer will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the predict_with_generate option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

The last argument to setup everything so we can push the model to the Hub regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the hub_model_id argument to set the repo name (it needs to be the full name, including your namespace: for instance "sgugger/marian-finetuned-en-to-ro" or "huggingface/marian-finetuned-en-to-ro").

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [34]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [35]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [36]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/fanbyprinciple/opus-mt-en-ro-finetuned-en-to-ro into local empty directory.


In [37]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation.
***** Running training *****
  Num examples = 610320
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 38145


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.push_to_hub()

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("fanbyprinciple/f"{model_name}-finetuned-{source_lang}-to-{target_lang}")

## things to do for bengali script

1. I need to find bengali documentation and accurate translation dataset
2. convert that into dataset 
3. load the seq to seq model 
4. train.